## Exercise

We have seen how to compute the log-odds between liberal-conservative for each book. Given this information, we can try to estimate political leanings of students. You can do this by summing the log-odds of their favorite books. 

Steps: 
1. Create a table with the log-odds of the books. 
2. Join the table with the log-odds with the book preferences table.
3. Sum the log-odds score for each student.

Evaluation:
* You have students that have declared their political preferences as Liberal, Conservative, Very Liberal, Very Conservative. Examine the scores for these students, to check how well this technique works. The simplest way is to compute the average (mean) log-odds for students that fall into the different groups. Alternatively, you can try to plot the full distribution of scores.
* Calculate a score for each student that did not declare a political view but has listed Favorite Books.

Notes: 
* You can do the work in MySQL or in Pandas. If you decide to work purely in Pandas, the [`merge`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html#pandas.DataFrame.merge) command allows you to perform joins between dataframes, in way similar to SQL.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (15, 7)


In [2]:
conn_string_fb = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
    user='student',
    password='dwdstudent2015',
    host='db.ipeirotis.org',
    port=3306,
    db='facebook')
engine_fb = create_engine(conn_string_fb)

books = '''
SELECT B.Book, P.PoliticalViews, COUNT(*) AS cnt 
FROM Profiles P JOIN FavoriteBooks B ON B.ProfileID = P.ProfileId  
WHERE PoliticalViews IS NOT NULL AND B.Book IS NOT NULL 
      AND (PoliticalViews = 'Liberal' OR PoliticalViews = 'Conservative')
AND B.Book IN (
    SELECT Book 
    FROM FavoriteBooks B JOIN Profiles P ON B.ProfileID = P.ProfileId  
    WHERE (P.PoliticalViews = 'Liberal' OR P.PoliticalViews = 'Conservative')
    GROUP BY Book HAVING COUNT(DISTINCT P.ProfileID)>10
)
GROUP BY B.Book, P.PoliticalViews;
'''
df_books = pd.read_sql(books, con=engine_fb)
# Fill the NaN entries with the value 0 
dfp = df_books.pivot_table(
    index='Book', 
    columns='PoliticalViews', 
    values='cnt').fillna(0)

In [3]:
polviews = '''
SELECT PoliticalViews, COUNT(*) AS cnt 
FROM facebook.Profiles
GROUP BY PoliticalViews
'''
df_polviews = pd.read_sql(polviews, con=engine_fb).set_index('PoliticalViews')
liberals = df_polviews.at['Liberal','cnt']
conservatives = df_polviews.at['Conservative','cnt']
dfp["Liberal_perc"] = 100*(dfp["Liberal"] +1)  / liberals
dfp["Conservative_perc"] = 100*(dfp["Conservative"] +1)  / conservatives

In [4]:
dfp["lift_liberal"]          = dfp["Liberal_perc"] / dfp["Conservative_perc"]
dfp["lift_conservative"]     = dfp["Conservative_perc"]  / dfp["Liberal_perc"]
dfp["log_odds_liberal"]      = np.log(dfp["lift_liberal"])
dfp["log_odds_conservative"] = np.log(dfp["lift_conservative"])

In [5]:
# Calculate the log-odds score for each student, by summing the log-odds score for the books they like
books = '''
SELECT B.Book, P.PoliticalViews, COUNT(*) AS cnt 
FROM Profiles P JOIN FavoriteBooks B ON B.ProfileID = P.ProfileId  
WHERE PoliticalViews IS NOT NULL AND B.Book IS NOT NULL 
AND PoliticalViews IN ('Liberal', 'Conservative', 'Very Liberal', 'Very Conservative')
AND B.Book IN (
    SELECT Book 
    FROM FavoriteBooks B JOIN Profiles P ON B.ProfileID = P.ProfileId  
    WHERE PoliticalViews IN ('Liberal', 'Conservative', 'Very Liberal', 'Very Conservative')
    GROUP BY Book HAVING COUNT(DISTINCT P.ProfileID)>10
)
GROUP BY B.Book, P.PoliticalViews;
'''
df_books = pd.read_sql(books, con=engine_fb)
# Fill the NaN entries with the value 0 
dfp = df_books.pivot_table(
    index='Book', 
    columns='PoliticalViews', 
    values='cnt').fillna(0)

In [6]:
polviews = '''
SELECT PoliticalViews, COUNT(*) AS cnt 
FROM facebook.Profiles
GROUP BY PoliticalViews
'''
df_polviews = pd.read_sql(polviews, con=engine_fb).set_index('PoliticalViews')
liberals = df_polviews.at['Liberal','cnt']
conservatives = df_polviews.at['Conservative','cnt']
very_liberals = df_polviews.at['Very Liberal','cnt']
very_conservatives = df_polviews.at['Very Conservative','cnt']

dfp["Liberal_perc"] = 100*(dfp["Liberal"] +1)  / liberals
dfp["Conservative_perc"] = 100*(dfp["Conservative"] +1)  / conservatives
dfp["Very_Liberal_perc"] = 100*(dfp["Very Liberal"] +1)  / very_liberals
dfp["Very_Conservative_perc"] = 100*(dfp["Very Conservative"] +1)  / very_conservatives

In [7]:
dfp["lift_liberal"]          = dfp["Liberal_perc"] / dfp["Conservative_perc"]
dfp["lift_conservative"]     = dfp["Conservative_perc"]  / dfp["Liberal_perc"]
dfp["lift_very_liberal"]          = dfp["Very_Liberal_perc"] / dfp["Very_Conservative_perc"]
dfp["lift_very_conservative"]     = dfp["Very_Conservative_perc"]  / dfp["Very_Liberal_perc"]
dfp["log_odds_liberal"]      = np.log(dfp["lift_liberal"])
dfp["log_odds_conservative"] = np.log(dfp["lift_conservative"])
dfp["log_odds_very_liberal"]      = np.log(dfp["lift_very_liberal"])
dfp["log_odds_very_conservative"] = np.log(dfp["lift_very_conservative"])
dfp.head()

PoliticalViews,Conservative,Liberal,Very Conservative,Very Liberal,Liberal_perc,Conservative_perc,Very_Liberal_perc,Very_Conservative_perc,lift_liberal,lift_conservative,lift_very_liberal,lift_very_conservative,log_odds_liberal,log_odds_conservative,log_odds_very_liberal,log_odds_very_conservative
Book,,,,,,,,,,,,,,,,
100 Years Of Solitude,3.0,55.0,0.0,27.0,0.866739,0.427350,1.229688,0.598802,2.028169,0.493056,2.053579,0.486955,0.707133,-0.707133,0.719584,-0.719584
1984,32.0,286.0,2.0,104.0,4.442037,3.525641,4.611331,1.796407,1.259923,0.793699,2.566974,0.389564,0.231051,-0.231051,0.942728,-0.942728
A Brave New World,2.0,7.0,1.0,8.0,0.123820,0.320513,0.395257,1.197605,0.386318,2.588542,0.330040,3.029940,-0.951095,0.951095,-1.108543,1.108543
A Brief History Of Time,3.0,7.0,0.0,1.0,0.123820,0.427350,0.087835,0.598802,0.289738,3.451389,0.146684,6.817365,-1.238777,1.238777,-1.919473,1.919473
A Clockwork Orange,8.0,86.0,0.0,42.0,1.346541,0.961538,1.888450,0.598802,1.400402,0.714080,3.153711,0.317087,0.336760,-0.336760,1.148580,-1.148580


In [8]:

query='''
SELECT P.ProfileID, P.Name, B.Book, P.PoliticalViews
FROM Profiles AS P JOIN FavoriteBooks AS B ON B.ProfileID = P.ProfileID
WHERE PoliticalViews IS NOT NULL AND B.Book IS NOT NULL
AND PoliticalViews IN ('Liberal', 'Conservative', 'Very Liberal', 'Very Conservative')
AND B.Book IN (
    SELECT Book 
    FROM FavoriteBooks B JOIN Profiles P ON B.ProfileID = P.ProfileId  
    WHERE PoliticalViews IN ('Liberal', 'Conservative', 'Very Liberal', 'Very Conservative')
    GROUP BY Book HAVING COUNT(DISTINCT P.ProfileID)>10
)
'''
df_student = pd.read_sql(query, con=engine_fb)


In [9]:
df_merge = df_student.merge(dfp,how='left',left_on='Book',right_on='Book')
df_filter = df_merge[['ProfileID','Name','Book','log_odds_liberal','log_odds_conservative','log_odds_very_liberal','log_odds_very_conservative']]
df_filter.head()

,ProfileID,Name,Book,log_odds_liberal,log_odds_conservative,log_odds_very_liberal,log_odds_very_conservative
0,800004,Sunny Kim,Memoirs Of A Geisha,0.450704,-0.450704,1.194042,-1.194042
1,800004,Sunny Kim,Middlesex,1.469273,-1.469273,1.357672,-1.357672
2,800004,Sunny Kim,Cat S Cradle,0.370661,-0.370661,0.754676,-0.754676
3,800004,Sunny Kim,Diary,-0.727951,0.727951,-0.666710,0.666710
4,800004,Sunny Kim,The Bible,-1.764870,1.764870,-2.612620,2.612620


In [10]:
df_filter.pivot_table(
    index=['ProfileID','Name'],
    values=['log_odds_liberal','log_odds_conservative','log_odds_very_liberal','log_odds_very_conservative'],
    aggfunc={'log_odds_liberal': np.sum,
             'log_odds_conservative': np.sum,
             'log_odds_very_liberal': np.sum,
             'log_odds_very_conservative': np.sum}
)

,,log_odds_conservative,log_odds_liberal,log_odds_very_conservative,log_odds_very_liberal
ProfileID,Name,,,,
800004,Sunny Kim,0.202182,-0.202182,-0.027059,0.027059
800011,Charles Kerecz,5.646376,-5.646376,6.786919,-6.786919
800012,Josh Fern,0.390658,-0.390658,4.191196,-4.191196
800015,Sacha Kenton,-0.652074,0.652074,0.310035,-0.310035
800017,Kelly Quann,0.993639,-0.993639,-1.488915,1.488915
800019,Anne Klingeberger,-3.929843,3.929843,0.746158,-0.746158
800021,Angeline Hsu,-1.314891,1.314891,-0.295355,0.295355
800027,Dave Birinyi,-4.231636,4.231636,-0.166624,0.166624
800028,April Gu,-3.584603,3.584603,1.357092,-1.357092


In [11]:
# Calculate the mean log-odds score for students with different political views 
# (hint: Very Conservatives will be "out of order" compared to "Conservatives")
df_filter = df_merge[['PoliticalViews','log_odds_liberal','log_odds_conservative','log_odds_very_liberal','log_odds_very_conservative']]
df_filter.pivot_table(
    index='PoliticalViews',
    values=['log_odds_liberal','log_odds_conservative','log_odds_very_liberal','log_odds_very_conservative'],
    aggfunc={'log_odds_liberal': np.mean,
             'log_odds_conservative': np.mean,
             'log_odds_very_liberal': np.mean,
             'log_odds_very_conservative': np.mean}

)

,log_odds_conservative,log_odds_liberal,log_odds_very_conservative,log_odds_very_liberal
PoliticalViews,,,,
Conservative,0.225512,-0.225512,0.344011,-0.344011
Liberal,-0.299481,0.299481,0.114801,-0.114801
Very Conservative,0.126783,-0.126783,1.035147,-1.035147
Very Liberal,-0.321366,0.321366,0.050057,-0.050057
